In [15]:
import random
from games import *

random.seed()
class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if self.current_player == 1:
            pit_index = pit - 1
            if pit_index < self.p1_pits_index[0] or pit_index > self.p1_pits_index[1]:
                return False
        else:
            pit_index = self.p2_pits_index[1] - (pit - 1)
            if pit_index < self.p2_pits_index[0] or pit_index > self.p2_pits_index[1]:
                return False
        
        return self.board[pit_index] > 0
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        import random
        valid_pits = []
        for pit in range(1, self.pits_per_player + 1):
            if self.valid_move(pit):
                valid_pits.append(pit)
        return random.choice(valid_pits) if valid_pits else None
    
    def play(self, pit):
        """
        Simulated a play in Mancala
        """
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        
        if self.winning_eval():
            print("GAME OVER")
            return self.board
        
        self.moves.append((self.current_player, pit))
        
        if self.current_player == 1:
            pit_index = pit - 1
            my_pits_range = self.p1_pits_index
            my_mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index
            opponent_pits_range = self.p2_pits_index
        else:
            pit_index = self.p2_pits_index[1] - (pit - 1)
            my_pits_range = self.p2_pits_index
            my_mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index
            opponent_pits_range = self.p1_pits_index
        
        stones = self.board[pit_index]
        self.board[pit_index] = 0
        
        current_index = pit_index
        
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:
                continue
            self.board[current_index] += 1
            stones -= 1
        
        if current_index != my_mancala:
            if my_pits_range[0] <= current_index <= my_pits_range[1] and self.board[current_index] == 1:
                opposite_index = len(self.board) - 2 - current_index
                if self.board[opposite_index] > 0:
                    captured = self.board[current_index] + self.board[opposite_index]
                    self.board[current_index] = 0
                    self.board[opposite_index] = 0
                    self.board[my_mancala] += captured
            
        self.current_player = 2 if self.current_player == 1 else 1
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_pits_sum = sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1]+1])
        p2_pits_sum = sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1]+1])
        
        if p1_pits_sum == 0 or p2_pits_sum == 0:
            self.board[self.p1_mancala_index] += p1_pits_sum
            self.board[self.p2_mancala_index] += p2_pits_sum
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[i] = 0
            return True
        
        return False
    
    # def terminal_test(self, state):
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        val = []
        if (self.current_player == 1):
            for pit in self.board[self.p1_pits_index]: 
                if (self.valid_move(pit)): 
                    val.append(pit)
        if (self.current_player == 2):
            for pit in self.board[self.p2_pits_index]: 
                if (self.valid_move(pit)): 
                    val.append(pit)
        return val
    
def run_mancala_trials(numTrials):
    moves_per_game = []
    p1_wins = 0
    p2_wins = 0
    ties = 0

    num_trials = numTrials

    for i in range(num_trials):

        game = Mancala(pits_per_player=6, stones_per_pit=4)

        p1_moves_played = 0
        p2_moves_played = 0

        while not game.winning_eval():
            if game.current_player == 1:
                game.play(game.random_move_generator())
                p1_moves_played += 1
            else:
                game.play(game.random_move_generator())
                p2_moves_played += 1
        
        moves_per_game.append((p1_moves_played, p2_moves_played))

        if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
            p1_wins += 1
        elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
            p2_wins += 1

    # # results

    p1_win_percentage = p1_wins / num_trials * 100
    p2_win_percentage = p2_wins / num_trials * 100

    p1_avg_moves = sum([x for (x,y) in moves_per_game]) / num_trials
    p2_avg_moves = sum([y for (x,y) in moves_per_game]) / num_trials

    print(f"Trials: {numTrials}")

    print("Player 1 Statistics:")
    print(f"Win percentage: %{p1_win_percentage}")
    print(f"Loss percentage: %{p2_win_percentage}")
    print(f"Tie percentage: %{100 - p1_win_percentage - p2_win_percentage}")
    print(f"Average # of moves: {p1_avg_moves}")

    print("")

    print("Player 2 Statistics:")
    print(f"Win percentage: %{p2_win_percentage}")
    print(f"Loss percentage: %{p1_win_percentage}")
    print(f"Tie percentage: %{100 - p1_win_percentage - p2_win_percentage}")
    print(f"Average # of moves: {p2_avg_moves}")

run_mancala_trials(100)

Trials: 100
Player 1 Statistics:
Win percentage: %40.0
Loss percentage: %52.0
Tie percentage: %8.0
Average # of moves: 22.58

Player 2 Statistics:
Win percentage: %52.0
Loss percentage: %40.0
Tie percentage: %8.0
Average # of moves: 22.14


**Questions:**
- **Is there a first move advantage? If so, how much?:** Yes there is, 2-5% which can be seen in the numbers. It converges asmtoptically, as you run more trials.

**Progress so Far:**
- We discovered you need more than 100 trials to accurately depict the first move advantage, also that the first player takes half a move more on avg.
- Our random players accurately play the game, don't make errors, although they currently make moves totally randomly, obviously, although in the future they will do so with intent. Some of the seeds are bad and *do* result in +20% tie rates so we opted to use the empty seed 'random.seed()' in order to consistently get different results. Hypothetically the first player will have an even larger win chance once the agents are operating with a *Minimax*. 
- Initially we saw that the number of moves each player was taking varied by greater than 1 in some trials, so we realized that we had accidentally implemented the rule that allows a player to repeat their turn. After fixing this, we saw that the number of moves between players differed by at most 1 move per each game.